# Managed Online Endpoint Logs and Metrics Integration with Log Analytics / Application Insights 

Learn how to capture logs and metrics from a Managed Online Endpoint using Application Insights and Log Analytics. AzureML Inference Server has extensive logging integrations with Application Insights that can be enabled with minimal changes to existing scoring logic. 

In this example we connect to an existing ['Workspace-based Application Insights Resource'](https://learn.microsoft.com/en-us/azure/azure-monitor/app/create-workspace-resource) explore the logging capabilities of AzureML Inference Server including the following: 
    - Request Logs
    - Model Data Logs
    - Exception Logs
    - Print Hook Logging (Stdout/Stderr)
    - Simple Custom Logging 

## Prerequisites
- You must have a Log Analytics Workspace-based Application Insights to use Python SDKs to query Application Insights logs. If you do not have one or have an Application Insights (Classic) instance, please follow the instructions in the ['Workspace-based Application Insights Resources'](https://learn.microsoft.com/en-us/azure/azure-monitor/app/create-workspace-resource) guide.
- You must have the following Python packages installed:
    - `azure-mgmt-resource`: Access property values of Log Analytics and Application Insights resources.
    - `azure-monitor-query`: Issue queries to Log Analytics workspaces 

Install them with this code:

In [ ]:
%pip install azure-mgmt-resource
%pip install azure-monitor-query 

## 1. Connect to Azure Machine Learning Workspace
### 1.1. Import the required libraries

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.monitor.query import LogsQueryClient
from azure.mgmt.resource import ResourceManagementClient
from datetime import timedelta
from azure.identity import DefaultAzureCredential
from random import randint
import json, time

### 1.2 Set variables

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

# The fully qualified resource ID ("/subscription/...") of the Application Insights resource to use for logging. 
# If not specified, the default Application Insights resource associated with the workspace will be used.
alternative_app_insights_id = "<APP_INSIGHTS_ID>"

endpoint_name = f"endpt-{randint(0, 100000)}"

### 1.3 Get a handle to the workspace

In [ ]:
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

## 2. Connect to App Insights / Log Analytics

### 2.1 Get App Insights / Log Analytics Details
The `InstrumentationKey` will be passed to the deployment as the `AML_APP_INSIGHTS_KEY` environment variable. 

In [ ]:
workspace = ml_client.workspaces.get(workspace_name)

resource_client = ResourceManagementClient(credential=credential, subscription_id=subscription_id)

app_insights_resource_id = workspace.application_insights if alternative_app_insights_id == "<APP_INSIGHTS_ID>" else alternative_app_insights_id
app_insights_resource = resource_client.resources.get_by_id(app_insights_resource_id, "2015-05-01")
app_insights_instrumentation_key = app_insights_resource.properties.get("InstrumentationKey")

log_analytics_resource_id = app_insights_resource.properties.get("WorkspaceResourceId")
log_analytics_resource = resource_client.resources.get_by_id(log_analytics_resource_id, "2022-10-01")
log_analytics_customer_id = log_analytics_resource.properties.get("customerId")

### 2.2 Get Logs Query client

In [ ]:
logs_client = LogsQueryClient(credential=credential)

## 3. Create a Managed Online Endpoint

App Insights can be enabled for a deployment with the `app_insights_enabled` argument set to `true` and the environment variable `AML_APP_INSIGHTS_KEY` set to the instrumentation key of the Log Analytics workspace. 

Two additional environment variables toggle log detail:
- `AML_MODEL_DC_STORAGE_ENABLED` (default=`true`): Activates the `Model Data Log`
- `APP_INSIGHTS_LOG_RESPONSE_ENABLED` (default: `false`): Toggles the `Response Value` logging in the `Request Log`.

The different logs and their fields include: 
- **Request Log**: With every endpoint that is not the “/” health check endpoint, a log is created with information about the fields below. This is logged in the `AppRequests` table (Log Analytics) / `requests` table (App Insights):
    - Request id
    - Response value (Not logged when `APP_INSIGHTS_LOG_RESPONSE_ENABLED` is `false`)
    - Request id  
    - Client Request Id
    - Container Id
    - Request path
    - URL of request, including params  
    - Duration  
    - Success (True or false)
    - Start time  
    - Response code
    - Http method
- **Exception Log**: Writes telemetry when exception is encountered. The following details are logged. This output can be seen in the `exceptions` table from base image, along with other information:
  - Container ID
  - Request ID
  - Client Request Id
- **Model Data Log**: When the scoring function is run, a log is created about the model data with the following information from base images. This output is seen in the `trace`  table. For this logging to take place, MDC must also be enabled, with `AML_MODEL_DC_STORAGE_ENABLED`.
  - Container Id
  - Request Id
  - Client Request Id
  - Workspace Name
  - Service Name
  - Models
  - Input
  - Prediction

- **Print Hook**: 
  STDOUT/STDERR output including user print functions will be visible in the `trace` logs under `STDOUT`. 
  - All messages within the user run function with the request-id prefix automatically prepended as follows:
    - `04c6f58d-510f-4e3a-933e-60ac20f2707d,User run function invoked.`
  - Within the init function, a series of zeroes will be prepended as follows:
    - `00000000-0000-0000-0000-000000000000,User init function invoked.`
  

### 3.1 Create an endpoint


In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

### 3.2 Create the deployment

For the initial deployment `APP_INSIGHTS_LOG_RESPONSE_ENABLED` is set to false.

In [ ]:
deployment = ManagedOnlineDeployment(
    name = "blue",
    app_insights_enabled=True,
    endpoint_name=endpoint_name,
    model= Model(path="../model-1/model/sklearn_regression_model.pkl"),
    code_configuration=CodeConfiguration(
        code="app-insights/code",
        scoring_script="score.py"
    ),
    environment=Environment(
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
        conda_file="../model-1/environment/conda.yml"
    ),
    environment_variables={
        "AML_APP_INSIGHTS_KEY": app_insights_instrumentation_key,
        "AML_MODEL_DC_STORAGE_ENABLED": True,
        "APP_INSIGHTS_LOG_RESPONSE_ENABLED": False
    }, 
    instance_type="Standard_DS2_v2",
    instance_count=1,
)
deployment = ml_client.online_deployments.begin_create_or_update(deployment).result()

In [ ]:
endpoint.traffic = {"blue": 100}
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

##  3. Request Log
The Request Log is stored in the `AppRequests` (`requests`) table.

### 3.1 Without Log Response

In [ ]:
ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, deployment_name="blue", request_file="../model-1/sample-request.json")
time.sleep(60)

In [ ]:
query=f"AppRequests \
    | where Properties['url'] contains '{deployment.name}.{endpoint.name}' \
    | order by TimeGenerated desc \
    | take 1"

query_resp = logs_client.query_workspace(log_analytics_customer_id, query, timespan=timedelta(days=1))
table = query_resp.tables[0]

to_dict = lambda t, d=None, n=1: [{k:json.loads(v) if k in (d or {'Properties'}) else v for k,v in zip(t.columns,r)} for r in t.rows[0:n]]
to_dict(table)

### 3.2 With Log Response

In [ ]:
deployment.environment_variables['APP_INSIGHTS_LOG_RESPONSE_ENABLED'] = True
deployment = ml_client.online_deployments.begin_create_or_update(deployment).result()

ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, deployment_name="blue", request_file="../model-1/sample-request.json")
time.sleep(60)

In [ ]:
query=f"AppRequests \
    | where Properties['url'] contains '{deployment.name}.{endpoint.name}' \
    | order by TimeGenerated desc \
    | take 1"

query_resp = logs_client.query_workspace(log_analytics_customer_id, query, timespan=timedelta(days=1))
table = query_resp.tables[0]

to_dict(table)

### 4. Model Data Log
The Model Data Log is stored in the `AppTraces` (`trace`) table. 

In [ ]:
query="AppTraces \
    | order by TimeGenerated desc \
    | take 1"

query_resp = logs_client.query_workspace(log_analytics_customer_id, query, timespan=timedelta(days=1))
table = query_resp.tables[0]

to_dict(table)

## 5. Cleanup

### 5.1 Delete the Managed Online Endpoint 

In [ ]:
ml_client.online_endpoints.begin_delete(endpoint).wait()